## Project Mission

As a data analyst, I am given a database that is comprisied of several music artists, albums, and tracks.  I have been tasked with answering questions for a record label about which music should be appropriate to add to their music store.  To analyze this, I have uploaded the database as a sqlite file and plan to put my SQL skills to the test.  This project will demonstrate my ability to join muliple tables together, use subqueries, and build aggregate functions.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///Chinook.sqlite

The record label wants me to analyze which type of music is popular in the USA.  To begin, I will start by familiarizing myself with their database.  Using an inner join, I will retrieve all the customers in the USA and their total amount spent on music purchases.

In [2]:
%%sql
SELECT c.CustomerId, c.FirstName, c.LastName, c.Country, "$" || "" || SUM(i.Total) AS Total_Spent
    FROM Customer c
    INNER JOIN Invoice i ON c.CustomerId = i.CustomerId
    WHERE c.Country IN ('USA') 
    GROUP BY c.CustomerId;

 * sqlite:///Chinook.sqlite
Done.


CustomerId,FirstName,LastName,Country,Total_Spent
16,Frank,Harris,USA,$37.62
17,Jack,Smith,USA,$39.62
18,Michelle,Brooks,USA,$37.62
19,Tim,Goyer,USA,$38.62
20,Dan,Miller,USA,$39.62
21,Kathy,Chase,USA,$37.62
22,Heather,Leacock,USA,$39.62
23,John,Gordon,USA,$37.62
24,Frank,Ralston,USA,$43.62
25,Victor,Stevens,USA,$42.62


In [3]:
%%sql
SELECT i.BillingCountry AS Country, "$" || "" || ROUND(SUM(i.Total), 2) AS Total_Spent
    FROM Invoice i
    WHERE i.BillingCountry IN ('USA')
    GROUP BY i.BillingCountry;

 * sqlite:///Chinook.sqlite
Done.


Country,Total_Spent
USA,$523.06


Using this database, write a query shows the most popular genres in the USA based on the number of tracks sold.

In [4]:
%%sql
WITH Tracks_Playlist AS
   (
    SELECT il.* FROM InvoiceLine il
        INNER JOIN Invoice i on il.InvoiceId = i.InvoiceId
        INNER JOIN Customer c on i.CustomerId = c.CustomerId
        INNER JOIN Track t on t.TrackId = il.TrackId
        INNER JOIN Genre g on g.GenreId = t.GenreId
            WHERE c.Country = "USA"
            AND g.Name <> "Comedy"
            AND g.Name <> "TV Shows"
            AND g.Name <> "Drama"
   )

SELECT
    g.Name Genre,
    COUNT(tp.InvoiceLineId) Tracks_Sold,
    ROUND(CAST(COUNT(tp.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) From Tracks_Playlist
    ), 4) || "%" Percentage_Sold
    
FROM Tracks_Playlist tp
    INNER JOIN Track t on t.TrackId = tp.TrackId
    INNER JOIN Genre g on g.GenreId = t.GenreId
        GROUP BY Genre
        ORDER BY Tracks_Sold DESC
        LIMIT 5;

 * sqlite:///Chinook.sqlite
Done.


Genre,Tracks_Sold,Percentage_Sold
Rock,157,0.3369%
Latin,91,0.1953%
Metal,64,0.1373%
Alternative & Punk,50,0.1073%
Jazz,22,0.0472%


It appears Rock is overwhelmingly popular in this music store.  However, Latin has a sizable following as well.

Write a query that breaksdown the top album selling albums in the USA based on number of tracks sold within those albums.

In [5]:
%%sql
WITH Tracks_Playlist AS
   (
    SELECT il.* FROM InvoiceLine il
        INNER JOIN Invoice i on il.InvoiceId = i.InvoiceId
        INNER JOIN Customer c on i.CustomerId = c.CustomerId
        INNER JOIN Track t on t.TrackId = il.TrackId
        INNER JOIN Genre g on g.GenreId = t.GenreId
            WHERE c.Country = "USA"
            AND g.Name <> "Comedy"
            AND g.Name <> "TV Shows"
            AND g.Name <> "Drama"
   )

SELECT
    ar.Name Artist,
    al.Title Album,
    g.Name Genre,
    COUNT(tp.InvoiceLineId) Tracks_Sold,
    ROUND(CAST(COUNT(tp.InvoiceLineId) AS FLOAT) / (
        SELECT COUNT(*) From Tracks_Playlist
    ), 4) || "%" Percentage_Sold
    
FROM Tracks_Playlist tp
    INNER JOIN Track t on t.TrackId = tp.TrackId
    INNER JOIN Genre g on g.GenreId = t.GenreId
    INNER JOIN Album al on al.AlbumId = t.AlbumId
    INNER JOIN Artist ar on ar.ArtistId = al.ArtistId
        GROUP BY Album
        ORDER BY Tracks_Sold DESC
        LIMIT 10;

 * sqlite:///Chinook.sqlite
Done.


Artist,Album,Genre,Tracks_Sold,Percentage_Sold
Caetano Veloso,Prenda Minha,Latin,11,0.0236%
Eric Clapton,Unplugged,Blues,11,0.0236%
Antônio Carlos Jobim,Chill: Brazil (Disc 2),Latin,10,0.0215%
Amy Winehouse,Back to Black,R&B/Soul,9,0.0193%
Green Day,International Superhits,Alternative & Punk,8,0.0172%
Soundgarden,A-Sides,Rock,7,0.015%
U2,B-Sides 1980-1990,Rock,7,0.015%
Toquinho & Vinícius,Vinícius De Moraes - Sem Limite,Bossa Nova,7,0.015%
Metallica,Black Album,Metal,6,0.0129%
Tim Maia,Serie Sem Limite (Disc 1),Latin,6,0.0129%


The customer have a heavy interest in Rock, Latin, and Metal genres.  The albums that had the most downloaded tracks were from artists Caetano Velosoand and Eric Clapton.  Based on this data, I would recommend the company to purchase "Prenda Minha" from Caetano Veloso, "A-Sides" from Soundgarden, and "B-Sides 1980-1990" from U2.

Write a query that shows the top selling sales support representatives within the company.

In [6]:
%%sql
WITH Total_Sales AS
    (
     SELECT
         i.CustomerId,
         c.SupportRepId,
         SUM(i.Total) AS Total
     FROM Invoice i
     INNER JOIN Customer c ON i.CustomerId = c.CustomerId
     GROUP BY 1, 2
    )

SELECT
    e.FirstName || " " || e.LastName Employee,
    e.Title AS Position,
    e.HireDate AS Hire_Date,
    "$" || "" || (SUM(ts.Total)) AS Total_Sales
FROM Total_Sales AS ts
INNER JOIN employee AS e ON e.EmployeeId = ts.SupportRepId
GROUP BY Employee;

 * sqlite:///Chinook.sqlite
Done.


Employee,Position,Hire_Date,Total_Sales
Jane Peacock,Sales Support Agent,2002-04-01 00:00:00,$833.04
Margaret Park,Sales Support Agent,2003-05-03 00:00:00,$775.4
Steve Johnson,Sales Support Agent,2003-10-17 00:00:00,$720.16


The data shows that although Jane Peacock has sold the most as a sales support agent.  However, Margaert Park has been the most productive agent because she was highered a year later and isn't far behind in Jane's total sales.

Analyzing the rest of the database, write a query that breaksdown the summary statistics and purchases of all customers around the world.

In [7]:
%%sql
WITH other_countries AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT COUNT(*)
                 FROM Customer
                 WHERE Country = c.Country
                ) = 1 THEN "Other"
           ELSE c.Country
       END AS Country,
       c.CustomerId,
       il.*
     FROM InvoiceLine il
     INNER JOIN Invoice i ON i.InvoiceId = il.InvoiceId
     INNER JOIN Customer c ON c.CustomerId = i.CustomerId
    )

SELECT
    Country,
    Customers,
    Total_Sales,
    Avg_Order,
    Avg_Sales_Per_Customer
FROM
    (
    SELECT
        Country,
        COUNT(DISTINCT CustomerId) Customers,
        "$" || "" || (ROUND(SUM(UnitPrice), 2)) Total_Sales,
        "$" || "" || (ROUND(SUM(UnitPrice) / COUNT(DISTINCT CustomerId), 2)) Avg_Sales_Per_Customer,
        "$" || "" || (ROUND(SUM(UnitPrice) / COUNT(DISTINCT InvoiceId), 2)) Avg_Order,
        CASE
            WHEN Country = "Other" THEN 1
            ELSE 0
        END AS Sort
    FROM other_countries
    GROUP BY Country
    ORDER BY Total_Sales ASC
    );

 * sqlite:///Chinook.sqlite
Done.


Country,Customers,Total_Sales,Avg_Order,Avg_Sales_Per_Customer
United Kingdom,3,$112.86,$5.37,$37.62
Germany,4,$156.48,$5.59,$39.12
Brazil,5,$190.1,$5.43,$38.02
France,5,$195.1,$5.57,$39.02
Canada,8,$303.96,$5.43,$38.0
USA,13,$523.06,$5.75,$40.24
Other,15,$604.3,$5.76,$40.29
India,2,$75.26,$5.79,$37.63
Portugal,2,$77.24,$5.52,$38.62
Czech Republic,2,$90.24,$6.45,$45.12


Management is trying to decide whether purchasing only the most popular tracks from music artists is better than purchasing the entire album.  Write a query that breaksdown how many customers actually bought the entire album.  

In [8]:
%%sql
WITH original_track AS
    (
     SELECT
         il.InvoiceId InvoiceId,
         MIN(il.TrackId) OrigTrackId
             FROM InvoiceLine il
             GROUP BY InvoiceId
    )

SELECT album_purchase AS Album_Purchase,
    COUNT(InvoiceId) AS Total_Invoices,
    ROUND(CAST(count(InvoiceId) AS FLOAT) / (
                                       SELECT COUNT(*) FROM Invoice
                                      ), 5)  || "%" AS Percentage
FROM
    (
    SELECT ot.*, CASE WHEN
                 (
                  SELECT t.TrackId FROM track t
                  WHERE t.AlbumId = (
                                      SELECT t2.AlbumId FROM track t2
                                      WHERE t2.TrackId = ot.OrigTrackid
                                     ) 

                  EXCEPT 

                  SELECT il2.TrackId FROM InvoiceLine il2
                  WHERE il2.InvoiceId = ot.InvoiceId
                 ) IS NULL
                    
                    AND
                 (
                  SELECT il2.TrackId FROM InvoiceLine il2
                  WHERE il2.InvoiceId = ot.InvoiceId

                  EXCEPT 

                  SELECT t.TrackId FROM track t
                  WHERE t.AlbumId = (
                                      SELECT t2.AlbumId FROM track t2
                                      WHERE t2.TrackId = ot.OrigTrackId
                                     ) 
                 ) IS NULL
             THEN "Yes"
             ELSE "No"
         END AS "Album_Purchase"
     FROM original_track ot
    )
GROUP BY album_purchase;

 * sqlite:///Chinook.sqlite
Done.


Album_Purchase,Total_Invoices,Percentage
No,410,0.99515%
Yes,2,0.00485%
